In [26]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import recall_score, precision_score, f1_score

In [27]:
BOM = 'DataSource/AB18MJ1_032_BOM.xlsx'
Matrix = 'Matrix/fabric_matrix.csv'

In [28]:
bom = pd.read_excel(BOM)

In [404]:
bom

,AIRBLASTER,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Style Number: AB18MJ1_032,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,NaN,NaN,NaN,NaN,NaN,Designer: JG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Season: Winter 2017/18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Insulation:,100gm in the body hood sleeves,NaN,Collection: Freedom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Style Name:,NaN,Seams:,Critically Taped,NaN,Vendor: Soluna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Toaster Jacket,NaN,MM Rating:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,3,ZIPPER DIRECTION:,"CF has right hand slider, pocket zippers close...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,4,BARTACK:,At stress points,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,5,HEAT CUT:,"Elastic, webbing, taffeta",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,6,GROMMETS/SNAPS:,Should have pellon backing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
matrix = pd.read_csv(Matrix, index_col = 0)

In [74]:
matrix[0:3].index

Int64Index([0, 1, 2], dtype='int64')

In [30]:
vectorizer = CountVectorizer(stop_words = 'english')

In [31]:
all_features = vectorizer.fit_transform(matrix.material)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(all_features, matrix.fabric, test_size = 0.3)

In [50]:
classifier = MultinomialNB()

In [51]:
classifier.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [52]:
nr_correct = (y_test == classifier.predict(X_test)).sum()
nr_correct

2934

In [53]:
nr_incorrect = y_test.size - nr_correct
nr_incorrect

65

In [54]:
fraction_wrong = nr_incorrect / (nr_correct + nr_incorrect)
fraction_wrong

0.021673891297099034

In [55]:
accuracy = 1 - fraction_wrong
accuracy

0.9783261087029009

In [56]:
classifier.score(X_test, y_test)

0.9783261087029009

In [57]:
recall_score(y_test, classifier.predict(X_test))

0.9649122807017544

In [58]:
precision_score(y_test, classifier.predict(X_test))

0.933641975308642

In [59]:
f1_score(y_test, classifier.predict(X_test))

0.9490196078431373

# Classify the fabric in a bom

## The classify function
* Check each row and return the index, if it is classified as True

In [510]:
def classify_series(series):
    index_list = []
    for word in series:
        x = [word]
        judge = vectorizer.transform(x)
        if  classifier.predict(judge) == 1:
            index_list.append(series[series == word].index[0])
        
    return index_list

## Walk through bom
* Parse a xlsm of bom, turn the row into a col.
* Put the col to the classify function
* Return the index of row that is True

In [502]:
def check_index(xlsx_path):
    bom = pd.read_excel(xlsx_path)
    database = []
    
    for row in bom.index:
        row_str = str()
        for col in bom:
            row_str = row_str + ', ' + str(bom.at[row, col])
        database.append(row_str)
    
    col = pd.Series(database)
    index_list = classify_series(col)
    
    return col

In [503]:
index_list = classify_series(x)

In [505]:
index_list

[2, 9, 10, 11, 12, 13, 14, 15, 16, 20, 21, 22, 52, 53, 64]

In [509]:
bom.loc[index_list,:]

,AIRBLASTER,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Style Number: AB18MJ1_032,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
2,NaN,NaN,Insulation:,100gm in the body hood sleeves,NaN,Collection: Freedom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,FABRICS & LININGS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,A,Shell Fabric,SOTD109-C1-Y 100% Polyester 10K/8K 10K/8K +2-L...,Main body,NaN,VINTAGE BLACK,NaN,BLACK,DARK NAVY,AB18MJ1_032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,B,Shell Fabric,SOTD109-PR01-Y 100% Polyester 10K/8K 10K/8K +2...,Main body,NaN,NaN,DINOFLAGE,NaN,NaN,AB18MJ1_032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,C,Shell Fabric,SOTP069-C1-H 100% Polyester 10K/8K 15k/10k+2-L...,"Sleeves, hood",NaN,BLACK,BLACK,OXBLOOD,CAMEL,AB18MJ1_032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,D,Lining,210T-E EMB07 Tery Emboss Taffeta(Supplied by ...,"Inside lining, powder skirt, pocket bag",NaN,BLACK,BLACK,BLACK,BLACK,AB18MJ1_032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,E,Other,Velvety Tricot - SOTO048-1 (supplied by Solis),"Comfort chin guard, inner front pocket bag panel",NaN,BLACK,BLACK,BLACK,BLACK,AB18MJ1_032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,F,Other,100% poly MESH - SOTC025-1 (supplied by Solis),"vents, internal goggle pocket",NaN,BLACK,BLACK,BLACK,BLACK,AB18MJ1_032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,G,Other,nylon/elastic lycra - SONO029-1 (supplied by S...,powder skirt,NaN,BLACK,BLACK,BLACK,BLACK,AB18MJ1_032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,K,Other,100gm Insulation,"Main body, hood, sleeves",NaN,NaN,NaN,NaN,NaN,AB18MJ1_032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [439]:
g = pd.Series([])

In [440]:
g.append('gg')

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid